In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from definitions import ROOT_DIR
import seaborn as sns

In [2]:
def calculate_detected_intensities(df, threshold=0.8):
    '''
    Make a column with background corrected intensities for detected compounds, and 0s for not detected compounds
    Change any negative values to zero
    Also add detectability column, where compounds with prediction value above threshold=0.8 are labelled as detected (1)
    '''

    df['detectability'] = df.pred_val >= threshold
    vals = df.spot_intensity_bgr_corrected * df.detectability
    df['effective_intensity'] = np.clip(vals, 0, None)
    return df

def get_class_size(metadata, class_column):
    sizes = metadata[class_column].value_counts()
    metadata['class_size'] = [sizes[k] for k in metadata[class_column]]
    return metadata

In [3]:
p_root_dir = Path(ROOT_DIR).parents[0]
p_data = p_root_dir / "5_data_analysis"

# Metrics and Catboost predictions for all ions in their target wells
p_predictions = p_data / "all_predictions_curated_11-Dec-2021.csv"

# Dataset info (lab, matrix, polarity, m/z range, ids, etc.)
p_datasets = p_data / "Datasets_1Jul2022.csv"

# Classification
p_chem_class = p_data / "custom_classification_v2.csv"
p_pathways = p_data / "pathways_v2.csv"

In [4]:
# Load predictions, format neutral loss column
predictions = pd.read_csv(p_predictions, index_col=0)[['dataset_id', 
                                                       'name_short', 
                                                       'adduct', 
                                                       'neutral_loss', 
                                                       'pred_val',
                                                       'spot_intensity_bgr_corrected']]
predictions.neutral_loss.fillna('', inplace=True)

# Add dataset metadata 
datasets = pd.read_csv(p_datasets)
datasets_info = datasets.groupby('Dataset ID').first()[['Polarity', 'Matrix short', 'Matrix long', 'Slide code', 'EMBL']]
datasets_info['sample_name'] = datasets_info['Matrix short']

# Merge with predictions
df = pd.merge(predictions, datasets_info, left_on='dataset_id', right_on='Dataset ID', how='left')

# Filter to keep only datasets chosen for plots about matrix comparison
df = df[df.EMBL]

# only consider data of detected ions
df = calculate_detected_intensities(df, threshold=0.8)
data = df[df.detectability]

In [5]:
# Load classification, add class size info
chem_classification_file = pd.read_csv(p_chem_class, index_col='internal_id')
chem_class = get_class_size(chem_classification_file[['name_short', 'main_coarse_class']].drop_duplicates(), 
                            'main_coarse_class')
chem_subclass = get_class_size(chem_classification_file[['name_short', 'fine_class']], 
                              'fine_class')

pathway_classification_file = pd.read_csv(p_pathways, index_col='internal_id')
pathway_class= get_class_size(pathway_classification_file[['name_short', 'main_coarse_path']].drop_duplicates(),
                         'main_coarse_path')
pathway_subclass = get_class_size(pathway_classification_file[['name_short','fine_path']],
                                  'fine_path')

<ipython-input-2-d153a7fd2bbe>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['class_size'] = [sizes[k] for k in metadata[class_column]]


### Here you would also have relevant filtering steps

## Example for any axis values except class, subclass, pathway and pathway subclass

In [6]:
# Aggregate data from individual ions per metabolite ('name_short'), per dataset ('dataset_id') and axis values

# for example for these axes
X='adduct'
Y='Matrix short'

step1 = data.pivot_table(index=['dataset_id', 'name_short', X, Y],
            values=['effective_intensity', 'detectability'],
            aggfunc=
                {'effective_intensity':sum,
                'detectability':max})

step1.head()

detectability  \
dataset_id           name_short                adduct Matrix short                  
2021-06-18_10h37m54s 2-Oxoglutaric acid        +Cl    9AA                    True   
                                               -H     9AA                    True   
                     3-Hydroxyanthranilic acid +Cl    9AA                    True   
                                               -H     9AA                    True   
                                               [M]-   9AA                    True   

                                                                    effective_intensity  
dataset_id           name_short                adduct Matrix short                       
2021-06-18_10h37m54s 2-Oxoglutaric acid        +Cl    9AA                    151.037020  
                                               -H     9AA                  65553.206930  
                     3-Hydroxyanthranilic acid +Cl    9AA                     41.606110  
                                               -H     9AA                   5885.244730  
                                               [M]-   9AA                   2547.082945

In [7]:
# Aggregate data per dataset and axis values
# Calculate what fraction metabolites in this dataset were detected with a given X, Y axis value
# There are 172 metaboites in total

n_metabolites = df.name_short.nunique()
step2 = step1.groupby(['dataset_id', X, Y]).agg({
                                                'effective_intensity':'mean',
                                                'detectability': lambda x: sum(x)/n_metabolites}) 


step2.head()

effective_intensity  detectability
dataset_id           adduct Matrix short                                    
2021-06-18_10h37m54s +Cl    9AA                    752.467978       0.360465
                     -H     9AA                  98242.664881       0.779070
                     [M]-   9AA                   7726.594086       0.186047
2021-06-18_10h41m59s +Cl    CHCA                    80.200649       0.029070
                     -H     CHCA                 11340.135429       0.523256

In [8]:
# Finally, take the average of results of all datasets

step3 = step2.groupby([X, Y]).agg({
                                'effective_intensity' : 'mean',
                                'detectability' : 'mean'})

step3.rename(columns={'detectability':'fraction_detected'}, inplace=True)
step3['log10_intensity'] = np.log10(step3['effective_intensity']+1)
step3.head()

effective_intensity  fraction_detected  log10_intensity
adduct Matrix short                                                         
+Cl    9AA                    752.467978           0.360465         2.877065
       CHCA                    80.200649           0.029070         1.909560
       CMBT                    97.961136           0.081395         1.995465
       ClCCA                   15.033169           0.023256         1.205019
       DAN                    411.088459           0.215116         2.614990

## Example for when you have class in axes

- chem_class.main_chem_class are chemical classes
- chem_subclass.fine_class are chemical subclasses
- pathways.main_coarse_path is the main pathway
- pathway_subclass.fine_path is the pathway "subclass"

In [9]:
# Then I merge it with predictions
data = data.merge(chem_class, on='name_short', how='right')

In [10]:
# chem_class goes with 'main_coarse_class'
# chem_subclass goes with 'fine_class'
# pathway_class goes with 'coarse_path'
# pathway_subclass goes with 'fine_path'

In [11]:
# First step is to  aggregate per metabolite, dataset and axes values

# Let's use matrix and class as example axes values
X='Matrix short'
Y='main_coarse_class'

step1 = data.pivot_table(index=['dataset_id', 'name_short', X, Y],
            values=['effective_intensity', 'detectability', 'class_size'],
            aggfunc=
                {'effective_intensity':'sum',
                'detectability':'max',
                'class_size':'first'},
            fill_value=0)

step1.head()

class_size  \
dataset_id           name_short                Matrix short main_coarse_class                                     
2021-06-18_10h37m54s 2-Oxoglutaric acid        9AA          Carboxylic acids                                 19   
                     3-Hydroxyanthranilic acid 9AA          Amino acids, peptides, and analogues             47   
                     3-Phosphoglyceric acid    9AA          Carboxylic acids                                 19   
                     4,5-Dihydroorotic acid    9AA          Nucleosides, nucleotides, and analogues          25   
                     4-Hydroxyproline          9AA          Amino acids, peptides, and analogues             47   

                                                                                                     detectability  \
dataset_id           name_short                Matrix short main_coarse_class                                        
2021-06-18_10h37m54s 2-Oxoglutaric acid        9AA          Carboxylic acids                                  True   
                     3-Hydroxyanthranilic acid 9AA          Amino acids, peptides, and analogues              True   
                     3-Phosphoglyceric acid    9AA          Carboxylic acids                                  True   
                     4,5-Dihydroorotic acid    9AA          Nucleosides, nucleotides, and analogues           True   
                     4-Hydroxyproline          9AA          Amino acids, peptides, and analogues              True   

                                                                                                     effective_intensity  
dataset_id           name_short                Matrix short main_coarse_class                                             
2021-06-18_10h37m54s 2-Oxoglutaric acid        9AA          Carboxylic acids                                65704.243950  
                     3-Hydroxyanthranilic acid 9AA          Amino acids, peptides, and analogues             8473.933785  
                     3-Phosphoglyceric acid    9AA          Carboxylic acids                               164535.047480  
                     4,5-Dihydroorotic acid    9AA          Nucleosides, nucleotides, and analogues        110215.133760  
                     4-Hydroxyproline          9AA          Amino acids, peptides, and analogues             2378.876126

In [12]:
# Next, aggregare per dataset and axes values

step2 = step1.pivot_table(index=['dataset_id', X,Y],
                      values=['effective_intensity', 'detectability', 'class_size'],
                      aggfunc = {
                                'class_size':'first',
                                'effective_intensity':'mean', # only when considering only 'detected' data
                                'detectability':'sum'
                                },
                      fill_value=0)

step2['fraction_detected'] = step2.detectability / step2.class_size
step2.head()

class_size  \
dataset_id           Matrix short main_coarse_class                                  
2021-06-18_10h37m54s 9AA          Amines                                        12   
                                  Amino acids, peptides, and analogues          47   
                                  Carbohydrates                                 17   
                                  Carboxylic acids                              19   
                                  Lipids and lipid-like molecules               36   

                                                                        detectability  \
dataset_id           Matrix short main_coarse_class                                     
2021-06-18_10h37m54s 9AA          Amines                                            2   
                                  Amino acids, peptides, and analogues             44   
                                  Carbohydrates                                    16   
                                  Carboxylic acids                                 17   
                                  Lipids and lipid-like molecules                  27   

                                                                        effective_intensity  \
dataset_id           Matrix short main_coarse_class                                           
2021-06-18_10h37m54s 9AA          Amines                                         189.781329   
                                  Amino acids, peptides, and analogues         32267.906277   
                                  Carbohydrates                                77358.531285   
                                  Carboxylic acids                             70890.988938   
                                  Lipids and lipid-like molecules             141500.361618   

                                                                        fraction_detected  
dataset_id           Matrix short main_coarse_class                                        
2021-06-18_10h37m54s 9AA          Amines                                         0.166667  
                                  Amino acids, peptides, and analogues           0.936170  
                                  Carbohydrates                                  0.941176  
                                  Carboxylic acids                               0.894737  
                                  Lipids and lipid-like molecules                0.750000

In [13]:
# Finally, take the average of results of all datasets

step3 = step2.groupby([X, Y]).agg({
                                'effective_intensity' : 'mean',
                                'fraction_detected' : 'mean'})

step3['log10_intensity'] = step3['effective_intensity'].apply(lambda x: np.log10(x+1))
step3.head()

effective_intensity  \
Matrix short main_coarse_class                                           
9AA          Amines                                       52289.929304   
             Amino acids, peptides, and analogues         19910.264315   
             Carbohydrates                                39260.854681   
             Carboxylic acids                             35691.948357   
             Lipids and lipid-like molecules             109539.210344   

                                                   fraction_detected  \
Matrix short main_coarse_class                                         
9AA          Amines                                         0.500000   
             Amino acids, peptides, and analogues           0.765957   
             Carbohydrates                                  0.705882   
             Carboxylic acids                               0.736842   
             Lipids and lipid-like molecules                0.763889   

                                                   log10_intensity  
Matrix short main_coarse_class                                      
9AA          Amines                                       4.718426  
             Amino acids, peptides, and analogues         4.299099  
             Carbohydrates                                4.593971  
             Carboxylic acids                             4.552582  
             Lipids and lipid-like molecules              5.039574